<a href="https://colab.research.google.com/github/Sanskar6877/BTP/blob/main/sbic_valid_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas tensorflow transformers scikit-learn
import pandas as pd
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from google.colab import files
import json

# Choose the file from your local machine
uploaded = files.upload()

# Read the JSON file with explicit encoding specification
json_filename = list(uploaded.keys())[0]
with open(json_filename, 'r', encoding='utf-8', errors='replace') as file:
    data = json.load(file)

# Convert the loaded JSON data to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Create a binary 'hate' column based on the 'label_off' column
df['hate'] = df['label_off'].apply(lambda x: 0 if x == 'not offensive' else 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['post'], df['hate'], stratify=df['hate'])

# Tokenize the input data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the input data
tokenized_data_train = tokenizer(
    X_train.tolist(),
    return_tensors="tf",
    padding=True,
    truncation=True,
    max_length=128  # Adjust this value based on your sequence length requirements
)

tokenized_data_test = tokenizer(
    X_test.tolist(),
    return_tensors="tf",
    padding=True,
    truncation=True,
    max_length=128  # Adjust this value based on your sequence length requirements
)

# Prepare TF datasets
tf_dataset_train = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': tokenized_data_train['input_ids'],
        'token_type_ids': tokenized_data_train['token_type_ids'],
        'attention_mask': tokenized_data_train['attention_mask'],
    },
    y_train.values
))

tf_dataset_test = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': tokenized_data_test['input_ids'],
        'token_type_ids': tokenized_data_test['token_type_ids'],
        'attention_mask': tokenized_data_test['attention_mask'],
    },
    y_test.values
))

# Shuffle and batch the training dataset
tf_dataset_train = tf_dataset_train.shuffle(buffer_size=len(X_train)).batch(16)

# Load and compile the model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model.compile(optimizer=Adam(learning_rate=3e-5), loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(tf_dataset_train, epochs=10)


Saving sbic_valid.json to sbic_valid.json


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
219/219 [==============================] - 138s 417ms/step - loss: 0.7105 - accuracy: 0.4761
Epoch 2/10
219/219 [==============================] - 91s 415ms/step - loss: 0.5813 - accuracy: 0.6159
Epoch 3/10
219/219 [==============================] - 91s 415ms/step - loss: 0.5856 - accuracy: 0.5464
Epoch 4/10
219/219 [==============================] - 91s 417ms/step - loss: 0.5396 - accuracy: 0.6268
Epoch 5/10
219/219 [==============================] - 91s 416ms/step - loss: 0.4239 - accuracy: 0.5559
Epoch 6/10
219/219 [==============================] - 91s 415ms/step - loss: 0.3550 - accuracy: 0.4424
Epoch 7/10
219/219 [==============================] - 90s 413ms/step - loss: 0.2119 - accuracy: 0.4670
Epoch 8/10
219/219 [==============================] - 92s 418ms/step - loss: 0.4158 - accuracy: 0.3764
Epoch 9/10
219/219 [==============================] - 91s 414ms/step - loss: 0.2270 - accuracy: 0.4567
Epoch 10/10
219/219 [==============================] - 90s 412ms/step - 